In [16]:
##Exercice 1
from math import *
from sympy import *
import numpy as np
    
def integ_romberg(f,a,b,epsilon):
    liste=[] ## initialisation de la liste
    inter=(b-a) ## Intervalle initiale
    liste.append([(inter/2)*(f(a)+f(b))])## On ajoute les intervalles à la liste
    for i in range(1,epsilon+1): ## boucle sur la précision
        inter = inter/2 ## on divise l'intervalle
        sum = 0
        for k in range(1,2**i ,2):
            sum = sum + f(a+k*inter) ## méthode des trapèzes
        row1 = [0.5*liste [i-1][0] + sum*inter]
        for j in range(1, i+1):
            row2 = row1[j-1] + (row1[j-1]+liste[i-1][j-1])/(4**j-1)
            row1.append(row2)
        liste.append(row1)
    return r

 
x=Symbol('x')
f = 3*x**2 + x**2 - 5 ## On définit la fonction f
print(integ_romberg(f,5,10,1e-6))



TypeError: 'Add' object is not callable

In [ ]:
#Exercice 3



In [ ]:
## Sources

# Pour l'exercice1, aide de https://mathsci2.appstate.edu/~wmcb/Class/4310/ClassNotes131/Romberg_Program_Listings.pdf